In [1]:
import pandas as pd 
import numpy as np
import seaborn as sns
from pylab import rcParams
from IPython.display import Audio


rcParams['figure.figsize'] = 14, 8

In [2]:
#FILE_PATH ='/Users/aziiz/Documents/Keyword_spotting/data_17042021/clips.xlsx'
#Added by: @Derguene
ROOT = '../../'
FILE_PATH = ROOT+'data/raw/clips.xlsx'

In [3]:
#Added by: @Aziz
data = pd.read_excel(FILE_PATH)
data.head()

,id,client_id,path,sentence,original_sentence_id,created_at,bucket,locale_id
0,1,0e183bb3-5449-48d5-b5b8-9c0b19d55f02,0e183bb3-5449-48d5-b5b8-9c0b19d55f02/e9e9661cb...,O ɓox,e9e9661cbf9855949c094f6de4001dc1b19ed761c2ea5c...,2021-01-15 10:44:05,train,6
1,2,0e183bb3-5449-48d5-b5b8-9c0b19d55f02,0e183bb3-5449-48d5-b5b8-9c0b19d55f02/02a670ba9...,Ƥetaa-naxak,02a670ba96f24cf45750adc406799c79b3e03f7dc76d97...,2021-01-15 10:44:07,train,6
2,3,0e183bb3-5449-48d5-b5b8-9c0b19d55f02,0e183bb3-5449-48d5-b5b8-9c0b19d55f02/dda43373b...,A ñaaƴ,dda43373b34abed8f54a694a161240e010488d17ad6bf9...,2021-01-15 10:44:09,train,6
3,4,0e183bb3-5449-48d5-b5b8-9c0b19d55f02,0e183bb3-5449-48d5-b5b8-9c0b19d55f02/c0cd07156...,Tadik,c0cd0715685c2ce27277b01f4f6a141027bc56ebf3b6f4...,2021-01-15 10:44:11,train,6
4,5,0e183bb3-5449-48d5-b5b8-9c0b19d55f02,0e183bb3-5449-48d5-b5b8-9c0b19d55f02/9cc6f1ad7...,Xarɓaxay,9cc6f1ad7b4559ccd62a51ca6dde93ff82604eff3f696e...,2021-01-15 10:44:14,train,6


In [4]:
#Added by: @Aziz
local_id = data['locale_id'].unique()
local_id

array([6, 7, 3, 5, 4, 8])

# Organizing audio files by language
We will create a folder for each language and store the corresponding records there

In [5]:
#Added by: @Derguene
# local_id corresponds to the languages ids, cf data/Readme
id2lang = {3: 'Diola',
           4: 'Mandingue',
           5: 'Poular',
           6: 'Sérère',
           7: 'Wolof',
           8: 'Soninké'}

In [6]:
#Added by: @Derguene
# Create a directory for each language
import os

TARGET_DIR = ROOT+'data/intermediate/'
cp = 0
for language_dir in id2lang.values():
    os.mkdir(TARGET_DIR+language_dir)
    cp+=1
print(f"{cp} directories successfully created!")

6 directories successfully created!


In [7]:
# You'll need to grant writing to newly created directories
#!sudo sudo chmod o+w $TARGET_DIR/*

In [8]:
#Added by: @Derguene
# Range audio data by language dir
from shutil import copy as copy_file
from tqdm import tqdm
cp = 0
err_msg = []

for src, locale_id in tqdm(data[['path', 'locale_id']].values):
    filename = '/'+id2lang[locale_id]+'_'+str(cp)
    src_path = ROOT+'data/raw/'+src
    tgt_path = TARGET_DIR+id2lang[locale_id]+filename

    try:
        copy_file(src_path, tgt_path)
        cp+=1
    except IOError as error:
        # [Errno 2] No such file or directory
        err_msg.append(str(error))
    
print(f"- {cp}/{len(data)} files successfully copied!")
print(f"- {len(err_msg)} files or directory not found!")

100%|███████████████████████████████████████████████████████████████████████████████| 24653/24653 [00:06<00:00, 3779.82it/s]

- 24385/24653 files successfully copied!
- 268 files or directory not found!


In [25]:
#Added by: @Derguene
# Recursively counting files in the source directory
# This will include the excel file and the 2 hidden files (don't take them into account)
!find $ROOT/data/raw/ -type f | wc -l

   24388


In [21]:
#Added by: @Derguene
# prints the file count per directory for the current directory level
!cd $TARGET_DIR && find . -type f | cut -d/ -f2 | sort | uniq -c

   1 .DS_Store
   1 .gitkeep
1928 Diola
2440 Mandingue
4623 Poular
 440 Soninké
2056 Sérère
12898 Wolof


In [26]:
#Added by: @Derguene
# Recursively counting files in the target directory (total)
# This will include the 2 hidden files (don't take them into account)
!find $TARGET_DIR -type f | wc -l

   24387


## Next steps:
* Process files/folders referenced by the `clips.xlsx` file and not found in the raw data
* Identify keywords by language
* Arrange files by keywords within languages

# Keywords exploration

In [50]:
#Added by: @Aziz
# Let's get the number of unique keywords
#keywords = data['sentence'].unique()
#Proposition: @Derguene
keywords = data.drop_duplicates(subset='sentence')
print(f'There are {len(keywords)} unique keywords')

There are 415 unique keywords


In [53]:
#Added by: @Derguene
# Print keywords and their corresponding language ids
keywords_w_ids = keywords.groupby(["locale_id"], as_index=False)["sentence"].count()
keywords_w_ids

,locale_id,sentence
0,3,77
1,4,70
2,5,69
3,6,74
4,7,72
5,8,53


In [54]:
#Added by: @Derguene
# Map language ids to their corresponding languages
keywords_w_ids["locale_id"] = keywords_w_ids["locale_id"].map(id2lang)
keywords_w_ids.rename(columns={'locale_id': 'languages',
                                'sentence': 'keywords_nb'}, inplace=True)
keywords_w_ids

,languages,keywords_nb
0,Diola,77
1,Mandingue,70
2,Poular,69
3,Sérère,74
4,Wolof,72
5,Soninké,53


In [41]:
#Added by: @Aziz
# Plot the unique keywords
keywords

array(['O ɓox', 'Ƥetaa-naxak', 'A ñaaƴ', 'Tadik', 'Xarɓaxay', 'Milyaar',
       'Milyoŋ', 'Ndeyjoor', 'Juroom-ñett', 'Ñaar', 'Bálamuk ',
       'Sibaakiir ', 'Kamay ', 'Ceme', 'Búbaar', 'Juroom-ñaar',
       'Ub / Tëj', 'Déedet', 'Gararb', 'Fukk', 'Taxawal', 'Demal', 'Ak',
       'Ñeent', 'Junne', 'Lal', 'Xaj', 'Doxal', 'Ubbi / Tijji', 'Tus ',
       'Ci suuf', 'Jaay', 'Benn', 'Juroom', 'Jënd', 'Téemeer', 'Càmmooy',
       'Juroom-ñeent', 'Ci kow', 'Picc', 'Fanweer', 'Ñett', 'Waaw',
       'Ci ginnaaw', 'Ci kanam', 'Juroom-benn', 'Dow     ',
       'Teemedɗe ɗiɗi', 'Jeeɗiɗi                ', 'Uddude',
       'Ɗiɗi                  ', 'Lees               ',
       'Ndiga                  ', 'Soodde', 'Goo                  ',
       'Yahde          ', 'Solndu           ', 'Udditde',
       'Leɗki             ', 'Ñaamo               ', 'Teemedere',
       'Capanɗe tati', 'Alaa ', 'Rawaandu      ', 'Caggal               ',
       'Jeetati           ', 'Mbalndi        ', 'Yeeso            '

In [43]:
#Added by: @Aziz
# Let's get the distribution of keywords
#sns.countplot(data['sentence'])

## Next steps 
* Choose the keywords that we should use for training , they are over 400 keywords in this dataset
* Choose our base line model
  * A pretrained/random wav2vec 2.0 model 
  * Fine-tune our model on the keywords that we choose